In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# reads in i and z band magnitudes and errors
redshift, K0, Temperature, Lbol = np.loadtxt("accept_main.txt",skiprows=2,
                                             usecols=[3,4,7,8],unpack=True)

In [ ]:
plt.plot(Temperature,Lbol,'bo')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Temperature [keV]')
plt.ylabel(r'Luminosity [$10^{44}$ ergs/s]')


In [ ]:
plt.hist(Temperature,bins=20)

In [ ]:
plt.hist(Lbol,bins=20)

In [ ]:
from scipy.stats import linregress

logTemp = np.log10(Temperature[Lbol > 0.0])
logLbol = np.log10(Lbol[Lbol > 0.0])

vals_returned = linregress(logTemp, logLbol)

print(vals_returned)

In [ ]:
Temp_for_fit = np.linspace(0.5,20.0,50)
Lum_from_fit = (10**vals_returned[1]) * (Temp_for_fit**vals_returned[0])


In [ ]:
plt.plot(10**logTemp,10**logLbol,'bo')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Temperature [keV]')
plt.ylabel(r'Luminosity [$10^{44}$ ergs/s]')
plt.plot(Temp_for_fit,Lum_from_fit,'r--')

In [ ]:
plt.plot(logTemp,logLbol,'bo')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Temperature [keV]')
plt.ylabel(r'Luminosity [$10^{44}$ ergs/s]')
plt.plot(np.log10(Temp_for_fit),np.log10(Lum_from_fit),'r--')

In [ ]:
plt.hist(redshift,bins=20)

In [ ]:
## CONVENIENCE FUNCTIONS: these are needed for our fitting

def est_error(ydata,yguess,error):
    '''
    Takes in the observed data, our current step's estimated y-values 
    for the model, and our guess for the errors in the data.
    
    Returns sum-of-squares error assuming a 2-parameter model
    '''
    return ((ydata-yguess)**2/(2*error**2)).sum()/(ydata.size-2)


def est_model_vals(alpha,b,logT):
    '''
    Given x-values and our MCMC code's estimates for A and B, this returns 
    estimated y values that we can compare to the actul data .
    '''
    return b + alpha*logT

def prior(alpha, b):
    alpha0 = 6.0
    sigma_alpha = 3
    b0 = -3.0
    sigma_b = 3.0
    
    p_alpha = (2.0*np.pi*sigma_alpha**2)**-0.5 * np.exp( -(alpha-alpha0)**2/(2.0*sigma_alpha**2))

    #if alpha < 2.7:
    #    p_alpha = 0.0
    
    p_b = (2.0*np.pi*sigma_b**2)**-0.5 * np.exp( -(b-b0)**2/(2.0*sigma_b**2))
    return p_alpha*p_b


In [ ]:
## The actual MCMC code.
import numpy.random as npr

estimated_error = 0.2

# Initial guesses for A and B
# Outcome should be relatively insensitive to these guesses!
alpha_old = 3
b_old = -1

# Range of step sizes that we take (linearly in -dA...dA and -dB...dB).
# This has to be relatively small, but if you make it too small you don't get
# Good sampling of the probability distribution function.
dA=dB=0.01

# Total number of points we're going to sample (should be at least 10^4)
Npts = 100000
N_burn = 1000

# Initial model values and 'error' for our starting position
y_old = est_model_vals(alpha_old,b_old,logTemp)
err_old = est_error(logLbol,y_old,estimated_error)

#print("y_old:",y_old)
#print("err_old:",err_old)

# These lists keep track of our Markov Chain and errors so we can get our PDF later.
alpha_guess = []
b_guess = []
errors = []

iter_count = 0

plt.plot(10**logTemp,10**logLbol,'bo')
plt.plot(10**logTemp,10**y_old,'r-',linewidth=4)
plt.xscale('log')
plt.yscale('log')


'''
This loop is where the magic happens.  We generate a new guess at (A,B) and compare
it to our original values using the chi^2 value.  If the new guess is better, we automatically
step in that direction.  If the new guess is worse, we still sometimes step in that direction if
a random number in the range [0,1) is pess than the ratio of the chi^2 values for the old and new 
choices of (A,B). 
'''
for i in range(Npts):
    
    # Guess for our new point (A,B)
    alpha_new = alpha_old + npr.normal(0.0,dA)
    b_new = b_old + npr.normal(0.0,dB)

    # model values based on the new A and B values
    ynew = est_model_vals(alpha_new,b_new,logTemp)
    
    # sum of squares value comparing data and model given our estimate
    # of the errors in the data.
    err_new = est_error(logLbol, ynew, estimated_error)
    
    
    # acceptance probability - ratio of likelihoods for old and new data points.
    p_accept = np.exp(-err_new+err_old) *prior(alpha_new,b_new)/prior(alpha_old,b_old)

    #print('a,b,e,p:', alpha_new, b_new, err_new, p_accept)
    
    
    # if R < p_accept, we keep this point.  Otherwise, keep on moving!
    if npr.random() < p_accept:

        #print('a,b,enew,eold,p:', alpha_new, b_new, err_new, err_old, err_new-err_old,p_accept)

        alpha_old = alpha_new
        b_old = b_new
        err_old = err_new
        
        #plt.plot(10**logTemp,10**ynew,alpha=0.3)#,'r--')
        
        if iter_count > N_burn:
            alpha_guess.append(alpha_old)
            b_guess.append(b_old)
            errors.append(err_old)

    iter_count += 1

print("kept:",len(alpha_guess))

In [ ]:
plt.plot(alpha_guess,b_guess,'b-')#,A_user,B_user,'co',markersize=10)
#plt.plot(Aguess[-1],Bguess[-1],'gs',markersize=10)
plt.xlabel('alpha vals')
plt.ylabel('b vals')
plt.title('Markov Chain for estimate of (A,B)')
len(b_guess)

In [ ]:
from matplotlib.colors import LogNorm

cts,xbin,ybin,img = plt.hist2d(alpha_guess, b_guess, bins=60,norm=LogNorm())
#plt.plot(A_user,B_user,'co',markersize=10)

# use np.argwhere() to find the bin(s) with the max counts
vals=np.argwhere(cts==cts.max())

# use those to guess our best values of A, B.  If there are more than one bins with the max
# number of counts, we're just using the first one we come to.  (This is not the best idea, but
# it makes the point)
Abest = xbin[vals[0,0]]
Bbest = ybin[vals[0,1]]
plt.plot(Abest,Bbest,'wD',markersize=10)

plt.colorbar()
plt.xlabel('A vals')
plt.ylabel('B vals')
plt.title('2D Histogram of walker positions')

print("A, B best:",Abest,Bbest)

In [ ]:
Lum_from_bayes_fit = 10**Bbest * Temp_for_fit**Abest


In [ ]:
plt.plot(10**logTemp,10**logLbol,'bo')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Temperature [keV]')
plt.ylabel(r'Luminosity [$10^{44}$ ergs/s]')
plt.plot(Temp_for_fit,Lum_from_fit,'r-',linewidth=4)
plt.plot(Temp_for_fit,Lum_from_bayes_fit,'g-',linewidth=4)


In [ ]:

A2_T500, A2_Terr, A2_Lbol, A2_Lerr = np.loadtxt("ACCEPT2_sanitized.dat",skiprows=5,
                                             usecols=[0,1,2,3],unpack=True)

plt.plot(A2_T500,A2_Lbol,'bo')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Temperature [keV]')
plt.ylabel(r'Luminosity [ergs/s]')